# Alerts Helper

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/alerts_helper/alerts_helper.ipynb)

In [9]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp
from urllib.parse import urlparse
import requests
import json
from datetime import datetime

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

colnames_clean = {
    "nm": "alert name",
    "fami": "alert family",
    "info": "alert description",
    "ch_n": "chart name",
    "ctx": "chart context",
    "st": "alert status",
    "v": "alert latest value",
    "tr_v": "alert triggered value",
    "units": "alert units",
    "cl": "alert class",
    "cm": "alert component",
    #"src": "alert source configuration",
    "t": "alert instance as at timestamp",
    "tr_t": "alert triggered at timestamp",
    "tp": "alert type",
    "to": "alert send to group",
}

In [10]:
# inputs
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' 
room_id = 'd8a4e0c5-7c79-4145-900e-83a9f06fcb6a' # all nodes
#room_id = 'ae33f57b-b54e-4236-a6de-054da3f0a748' # machine learning room
output_dir = f'output/{space_id}/{room_id}'
now = datetime.now().strftime("%Y%m%d_%H%M%S")

In [11]:
def get_alerts_cloud(space_id, room_id, api_token=None, base_url='https://app.netdata.cloud', node_ids=[]):
    """
    """
    
    if api_token is None:
        api_token = os.getenv('NETDATA_API_TOKEN')
    
    base_url = 'https://app.netdata.cloud'
    url = f'{base_url}/api/v2/spaces/{space_id}/rooms/{room_id}/alerts'
    headers = {'Accept': '*/*', 'Content-Type': 'application/json', 'Authorization': f'Bearer {api_token}'}
    data = {"scope":{"nodes":[],"contexts":[]},"selectors":{"status":["raised"]},"options":["summary","values","instances"]}
    r = requests.post(url, headers=headers, data=json.dumps(data))
    
    if r.status_code != 200:
        
        print(f'Error: {r.status_code, r.text}')
        return None
    
    else:
            
        return r.json()
    

def make_prompt(df_alert_instances):
    prompt = f"""
    You are an experienced SRE and sysadmin. You are monitoring your infrastructure using Netdata Cloud.

    Below you are given a list of alert instances representing all active alerts in your Netdata Cloud space right now.

    Can you summarize the and assess to state of your nodes and infrastructure given the dictionary of active alerts.
    
    Please just condense it to a paragraph or two of the key and most important points.
    
    Provide a list of some potential next steps in order of priority (be as concrete as possible and provide examples where relevant or possible).
    
    Use markdown to format your response.

    Here is the list of active alert instances:

    {df_alert_instances.to_dict(orient='records')}
    
    """
    
    return prompt


In [12]:
alerts = get_alerts_cloud(space_id, room_id, api_token=NETDATA_API_TOKEN)

In [13]:
nodes_map = {n['ni']:n['nm'] for n in alerts['nodes']}

In [14]:
df_alert_instances = pd.DataFrame(alerts['alert_instances'])
df_alert_instances['node name'] = df_alert_instances['ni'].map(nodes_map)
df_alert_instances = df_alert_instances.rename(columns=colnames_clean, inplace=False)
df_alert_instances = df_alert_instances[['node name'] + list(colnames_clean.values())]
df_alert_instances['alert instance as at timestamp'] = pd.to_datetime(df_alert_instances['alert instance as at timestamp'], unit='s')
df_alert_instances['alert triggered at timestamp'] = pd.to_datetime(df_alert_instances['alert triggered at timestamp'], unit='s')
df_alert_instances

,node name,alert name,alert family,alert description,chart name,chart context,alert status,alert latest value,alert triggered value,alert units,alert class,alert component,alert instance as at timestamp,alert triggered at timestamp,alert type,alert send to group
0,redis-master-0,10min_cpu_usage,cpu,average CPU utilization over the last 10 minut...,system.cpu,system.cpu,WARNING,93.298802,87.823290,%,Utilization,CPU,2023-07-25 12:29:32,2023-07-25 09:49:30,System,sysadmin
1,postgresql-0,10min_cpu_iowait,cpu,average CPU iowait time over the last 10 minutes,system.cpu,system.cpu,WARNING,0.000000,46.320023,%,Utilization,CPU,1970-01-01 00:00:00,2023-07-21 09:42:22,System,sysadmin
2,postgresql-0,10min_disk_backlog,nvme2n1,average backlog size of the nvme2n1 disk over ...,disk_backlog.nvme2n1,disk.backlog,WARNING,8858.068964,5321.927521,ms,Latency,Disk,2023-07-25 12:29:32,2023-07-25 01:16:50,System,silent
3,postgresql-0,10min_disk_utilization,nvme2n1,average percentage of time nvme2n1 disk was bu...,disk_util.nvme2n1,disk.util,WARNING,99.766667,99.468333,%,Utilization,Disk,2023-07-25 12:29:32,2023-07-25 09:44:21,System,silent
4,postgresql-0,load_average_1,load,system one-minute load average,system.load,system.load,WARNING,14.980000,16.620000,load,Utilization,Load,2023-07-25 12:29:32,2023-07-25 11:52:32,System,sysadmin
5,postgresql-0,load_average_15,load,system fifteen-minute load average,system.load,system.load,WARNING,15.050000,8.690000,load,Utilization,Load,2023-07-25 12:29:32,2023-07-12 09:33:19,System,sysadmin
6,postgresql-0,load_average_5,load,system five-minute load average,system.load,system.load,WARNING,15.560000,8.010000,load,Utilization,Load,2023-07-25 12:29:32,2023-07-25 09:44:16,System,sysadmin
7,netdata-collectors-0,httpcheck_web_service_bad_content,status,percentage of HTTP responses from https://www....,httpcheck_SOAP_test.request_status,httpcheck.status,CRITICAL,100.000000,100.000000,%,Workload,HTTP endpoint,2023-07-25 12:30:02,2023-06-12 09:15:15,Web Server,webmaster
8,netdata-collectors-0,httpcheck_web_service_timeouts,status,percentage of timed-out HTTP requests to https...,httpcheck_POST_test.request_status,httpcheck.status,CRITICAL,95.000000,40.000000,%,Latency,HTTP endpoint,2023-07-25 12:30:02,2023-07-25 02:40:21,Web Server,webmaster
9,ml-demo-stable,ml_5min_system_ram,ram,rolling 5min anomaly rate for system.ram chart,system.ram,system.ram,WARNING,0.000000,8.000000,%,Anomaly,CPU,1970-01-01 00:00:00,2023-07-25 12:10:04,System,root


In [15]:
prompt = make_prompt(df_alert_instances)
print(prompt)


    You are an experienced SRE and sysadmin. You are monitoring your infrastructure using Netdata Cloud.

    Below you are given a list of alert instances representing all active alerts in your Netdata Cloud space right now.

    Can you summarize the and assess to state of your nodes and infrastructure given the dictionary of active alerts.
    
    Please just condense it to a paragraph or two of the key and most important points.
    
    Provide a list of some potential next steps in order of priority (be as concrete as possible and provide examples where relevant or possible).
    
    Use markdown to format your response.

    Here is the list of active alert instances:

    [{'node name': 'redis-master-0', 'alert name': '10min_cpu_usage', 'alert family': 'cpu', 'alert description': 'average CPU utilization over the last 10 minutes (excluding iowait, nice and steal)', 'chart name': 'system.cpu', 'chart context': 'system.cpu', 'alert status': 'WARNING', 'alert latest value': 93.

In [16]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

# call openai api
completion = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
)

reply_content = completion.choices[0].message.content
#pp.pprint(reply_content)

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 5667 tokens. Please reduce the length of the messages.

In [ ]:
print(reply_content)

Based on the list of active alert instances, it appears that there are multiple warnings across different components of the infrastructure. 

First, there is a warning for high CPU iowait time on the node 'ml-demo-nightly'. The average CPU iowait time over the last 10 minutes is 41.06%, which is above the triggered value of 41.12%.

Second, there are warnings for high disk backlogs on two disks ('ubuntu--vg-ubuntu--lv' and 'sda') on the same node. The average backlog size for both disks over the last 10 minutes is above the triggered values.

Third, there are warnings for high system load averages on the same node. The one-minute load average is 60.99, which is above the triggered value of 21.59. The five-minute load average is 61.06, which is above the triggered value of 11.01. The fifteen-minute load average is 41.77, which is above the triggered value of 4.29.

Lastly, there are multiple warnings for anomalies in different dimensions of CPU usage, memory usage, and processes. These 

In [ ]:
if not os.path.exists(output_dir):
   os.makedirs(output_dir)

file_name_prompt = f'{space_id}_{room_id}__{now}_PROMPT.txt'
with open(f'{output_dir}/{file_name_prompt}', 'w') as f:
    f.write(prompt)
    
file_name_result = f'{space_id}_{room_id}__{now}_RESULT.md'
with open(f'{output_dir}/{file_name_result}', 'w') as f:
    f.write(reply_content)
